# Pràctica 3 - PLH

### Realitzada pels alumnes Lluc Furriols i Pau Prat Moreno

In [3]:
'''
import os 
f = open("/dev/null", "w")
os.dup2(f.fileno(), 2)
f.close()

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()
'''

'\nimport os \nf = open("/dev/null", "w")\nos.dup2(f.fileno(), 2)\nf.close()\n\nimport nltk\nimport ssl\n\ntry:\n    _create_unverified_https_context = ssl._create_unverified_context\nexcept AttributeError:\n    pass\nelse:\n    ssl._create_default_https_context = _create_unverified_https_context\n\nnltk.download()\n'

## Importació de llibreries

In [4]:
import nltk
nltk.download('punkt', quiet=True) # Tokenitzador
nltk.download('averaged_perceptron_tagger', quiet=True) # Etiquetador POS
nltk.download('maxent_ne_chunker', quiet=True) # Etiquetador Entitats Anomenades
nltk.download('words', quiet=True)

True

## Carreguem les dades

In [5]:
nltk.download('conll2002')
from nltk.corpus import conll2002

train_esp = conll2002.iob_sents('esp.train') # Train, 
val_esp = conll2002.iob_sents('esp.testa') # Val
test_esp = conll2002.iob_sents('esp.testb') # Test

train_ned = conll2002.iob_sents('ned.train') # Train
val_ned = conll2002.iob_sents('ned.testa') # Val
test_ned = conll2002.iob_sents('ned.testb') # Test

[nltk_data] Downloading package conll2002 to /Users/pau/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


In [6]:
train_esp

[[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')], [('-', 'Fg', 'O')], ...]

In [7]:
def get_token(sequence):
    """
    Retorna una llista de tokens.
    """
    return [[(token) for token, pos, entity in sentence] for sentence in sequence]

def get_token_entity(sequence):
    """
    Retorna una llista de tokens i les seves entitats.
    """
    return [[(token, entity) for token, pos, entity in sentence] for sentence in sequence]

In [8]:
print(get_token_entity(train_esp)[0])

[('Melbourne', 'B-LOC'), ('(', 'O'), ('Australia', 'B-LOC'), (')', 'O'), (',', 'O'), ('25', 'O'), ('may', 'O'), ('(', 'O'), ('EFE', 'B-ORG'), (')', 'O'), ('.', 'O')]


In [9]:
import string

class FeatureGetter:
    """
    Aquesta classe s'utilitza per obtenir diferents característiques d'un token de text.
    """
    def __init__(self):
        pass

    def has_capitalization(self, token):
        """
        Comprova si el token conté alguna lletra majúscula.
        """
        return any(char.isupper() for char in token)

    def has_digit(self, token):
        """
        Comprova si el token conté algun dígit.
        """
        return any(char.isdigit() for char in token)

    def has_punctuation(self, token):
        """
        Comprova si el token conté algun signe de puntuació.
        """
        return any(char in string.punctuation for char in token)

    def get_prefix(self, token, n=3):
        """
        Obté el prefix del token amb una longitud de n caràcters.
        """
        return token[:n] if len(token) > n else token

    def get_suffix(self, token, n=3):
        """
        Obté el sufix del token amb una longitud de n caràcters.
        """
        return token[-n:] if len(token) > n else token

    def all_caps(self, token):
        """
        Comprova si totes les lletres del token són majúscules.
        """
        return token.isupper()

    def is_capitalized(self, token):
        """
        Comprova si la primera lletra del token és majúscula.
        """
        return token[0].isupper()

    def get_features(self, token, add_prefix_suffix=True):
        """
        Obté un diccionari amb les característiques del token.
        """
        features = {
            'bias': 1.0,
            'has_capitalization': self.has_capitalization(token),
            'has_digit': self.has_digit(token),
            'has_punctuation': self.has_punctuation(token),
            'all_caps': self.all_caps(token),
            'is_capitalized': self.is_capitalized(token),
        }

        if add_prefix_suffix:
            features['prefix'] = self.get_prefix(token)
            features['suffix'] = self.get_suffix(token)

        return features

# Exemple d'ús:
feature_getter = FeatureGetter()
token_features = feature_getter.get_features('Barcelona')
print(token_features)

{'bias': 1.0, 'has_capitalization': True, 'has_digit': False, 'has_punctuation': False, 'all_caps': False, 'is_capitalized': True, 'prefix': 'Bar', 'suffix': 'ona'}


## Codificació BIO

In [24]:
def extract_entities(tagged_words, encoding='BIO'):
    """
    Extreu les entitats d'una llista de paraules etiquetades segons l'encoding especificat.
    
    Arguments:
        tagged_words: una llista de tuples (word, tag), on 'word' és una paraula del text i 'tag' és la seva etiqueta BIO/BIOE/BIOW.
        encoding: el tipus de codificació utilitzat per les etiquetes ('BIO', 'BIOW', 'BIOE').
        
    Retorna:
        Una llista de tuples (start_index, end_index, entity_type) que representen les entitats trobades.
        'start_index' i 'end_index' són els índexs on comença i acaba l'entitat en la llista de paraules, i 'entity_type' és el tipus d'entitat.
    """

    entities = []  # Llista on guardarem les entitats trobades
    current_entity = []  # Guarda les paraules de l'entitat actual
    current_type = None  # Tipus de l'entitat actual
    current_start_index = None  # Índex d'inici de l'entitat actual

    for index, (word, tag) in enumerate(tagged_words):
        if tag == 'O':  # Si el tag és 'O', estem fora d'una entitat
            if current_entity:  # Si hi havia una entitat en construcció, l'afegim a la llista
                entities.append((current_start_index, index - 1, current_type))
                current_entity = []
                current_type = None
            continue

        tag_prefix = tag[:1]  # Obtenim el prefix del tag (B, I, W, E)
        tag_type = tag[2:]  # Obtenim el tipus de l'entitat del tag

        if encoding in ['BIO', 'BIOW', 'BIOE']:
            if tag_prefix in ['B', 'W']:  # Començament d'una nova entitat o entitat de paraula única
                if current_entity:
                    entities.append((current_start_index, index - 1, current_type))
                current_entity = [word]
                current_start_index = index
                current_type = tag_type
                if tag_prefix == 'W':  # Si és una entitat de paraula única, la tanquem immediatament
                    entities.append((current_start_index, index, current_type))
                    current_entity = []
                    current_type = None
            elif tag_prefix == 'I' and current_type == tag_type:  # Continuació de l'entitat actual
                current_entity.append(word)
            elif encoding == 'BIOE' and tag_prefix == 'E' and current_type == tag_type:  # Final d'entitat en BIOE
                current_entity.append(word)
                entities.append((current_start_index, index, current_type))
                current_entity = []
                current_type = None
            else:  # Si no coincideix el tipus o es troba una etiqueta inesperada, es reseteja l'entitat actual
                if current_entity:
                    entities.append((current_start_index, index - 1, current_type))
                    current_entity = []
                current_type = None
        else:
            raise ValueError("Encoding not supported")  # Si l'encoding no és suportat, llancem una excepció

    # Afegir l'última entitat si la llista no acaba en 'O'
    if current_entity:
        entities.append((current_start_index, index, current_type))

    return entities

tagged_words = get_token_entity(train_esp)[3]

entities = extract_entities(tagged_words)
print(entities)


[(3, 4, 'PER'), (13, 14, 'ORG'), (18, 18, 'LOC'), (20, 20, 'LOC'), (59, 59, 'MISC')]


In [21]:
get_token_entity(train_esp)[3]

[('La', 'O'),
 ('petición', 'O'),
 ('del', 'O'),
 ('Abogado', 'B-PER'),
 ('General', 'I-PER'),
 ('tiene', 'O'),
 ('lugar', 'O'),
 ('después', 'O'),
 ('de', 'O'),
 ('que', 'O'),
 ('un', 'O'),
 ('juez', 'O'),
 ('del', 'O'),
 ('Tribunal', 'B-ORG'),
 ('Supremo', 'I-ORG'),
 ('del', 'O'),
 ('estado', 'O'),
 ('de', 'O'),
 ('Victoria', 'B-LOC'),
 ('(', 'O'),
 ('Australia', 'B-LOC'),
 (')', 'O'),
 ('se', 'O'),
 ('viera', 'O'),
 ('forzado', 'O'),
 ('a', 'O'),
 ('disolver', 'O'),
 ('un', 'O'),
 ('jurado', 'O'),
 ('popular', 'O'),
 ('y', 'O'),
 ('suspender', 'O'),
 ('el', 'O'),
 ('proceso', 'O'),
 ('ante', 'O'),
 ('el', 'O'),
 ('argumento', 'O'),
 ('de', 'O'),
 ('la', 'O'),
 ('defensa', 'O'),
 ('de', 'O'),
 ('que', 'O'),
 ('las', 'O'),
 ('personas', 'O'),
 ('que', 'O'),
 ('lo', 'O'),
 ('componían', 'O'),
 ('podían', 'O'),
 ('haber', 'O'),
 ('obtenido', 'O'),
 ('información', 'O'),
 ('sobre', 'O'),
 ('el', 'O'),
 ('acusado', 'O'),
 ('a', 'O'),
 ('través', 'O'),
 ('de', 'O'),
 ('la', 'O'),
 ('página

## Codificació IO

In [11]:
'''from nltk.tag import CRFTagger
import pycrfsuite

ct = CRFTagger(feature_func=None)
#train and test sets without the postag
train_esp_bio = get_token_entity(train_esp)
test_esp_bio = get_token_entity(test_esp)
print(train_esp_bio[0])

#y_test is the true labels
y_test = [[iob for word, iob in sent] for sent in test_esp_bio]
print(y_test[0])

#train the model
ct.train(train_esp_bio, 'model.crf.tagger')

#predict the labels
y_pred = ct.tag_sents([[word for word, iob in sent] for sent in test_esp_bio])
y_pred = [[iob for word, iob in sent] for sent in y_pred]
print(y_pred[0])


#show the confusion matrix
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Flatten y_test and y_pred
y_test_flat = [iob for sent in y_test for iob in sent]
y_pred_flat = [iob for sent in y_pred for iob in sent]

# Generate confusion matrix
cm = confusion_matrix(y_test_flat, y_pred_flat)

# Visualize confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

#show the classification report
from sklearn.metrics import classification_report
print(classification_report(y_test_flat, y_pred_flat))
ct.accuracy(test_esp_bio)'''

"from nltk.tag import CRFTagger\nimport pycrfsuite\n\nct = CRFTagger(feature_func=None)\n#train and test sets without the postag\ntrain_esp_bio = get_token_entity(train_esp)\ntest_esp_bio = get_token_entity(test_esp)\nprint(train_esp_bio[0])\n\n#y_test is the true labels\ny_test = [[iob for word, iob in sent] for sent in test_esp_bio]\nprint(y_test[0])\n\n#train the model\nct.train(train_esp_bio, 'model.crf.tagger')\n\n#predict the labels\ny_pred = ct.tag_sents([[word for word, iob in sent] for sent in test_esp_bio])\ny_pred = [[iob for word, iob in sent] for sent in y_pred]\nprint(y_pred[0])\n\n\n#show the confusion matrix\nfrom sklearn.metrics import confusion_matrix\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n# Flatten y_test and y_pred\ny_test_flat = [iob for sent in y_test for iob in sent]\ny_pred_flat = [iob for sent in y_pred for iob in sent]\n\n# Generate confusion matrix\ncm = confusion_matrix(y_test_flat, y_pred_flat)\n\n# Visualize confusion 